In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, fixed, FloatSlider, IntSlider

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
def get_dados(indice='portfolio'):
    if indice=='portfolio':
        df = pd.read_csv('dados/retorno_portfolio.csv', parse_dates=['Date'])
        df.fillna(0, inplace=True)
    elif indice=='ACWI':    
        df = pd.read_csv('dados/retorno_portfolio.csv', parse_dates=['Date'])
        df.fillna(0, inplace=True)
        df['retorno'] = df['ACWI']
    else:
        url = f'https://query1.finance.yahoo.com/v7/finance/download/{indice}?period1=852076800&period2=1634743473&interval=1d&events=history&includeAdjustedClose=true'
        df = pd.read_csv(url)
        df['Date'] = pd.to_datetime(df['Date'])
        df['retorno'] = df['Close'].pct_change()
        
    df['year_week'] = df['Date'].dt.strftime('%Y-%U')
    df['retorno acumulado'] = df['retorno'] + 1
    df['retorno acumulado'] = df['retorno acumulado'].cumprod()
    
    df_gsrai = pd.read_csv('dados/GSRAII.csv', parse_dates=['Date']).sort_values('Date')

    df = df.merge(df_gsrai, on='Date')
    return df[['Date', 'retorno acumulado', 'GSRAII Index', 'year_week']].dropna()

### Indices:
- ^STOXX50E = *EURO STOXX 50*
- ^GDAXI = *DAX*
- ^FTSE = *FTSE 100*
- ACWI
- portfolio

In [6]:
# Trocar codigo do indice
indice='^STOXX50E'
df = get_dados()
df

,Date,retorno acumulado,GSRAII Index,year_week
0,1997-01-02,1.000000,-0.04,1997-00
1,1997-01-03,1.004739,0.13,1997-00
2,1997-01-06,1.013472,0.22,1997-01
3,1997-01-07,1.013524,0.21,1997-01
4,1997-01-08,1.012990,0.22,1997-01
...,...,...,...,...
6385,2021-08-19,2.640252,-0.11,2021-33
6386,2021-08-20,2.627207,-0.09,2021-33
6387,2021-08-23,2.677371,-0.02,2021-34
6388,2021-08-24,2.686924,0.08,2021-34


In [ ]:
#%% Semanal
df_gp = df.groupby('year_week').agg('last')

for pct in range(1,6):
    df_gp[f'pct_{pct}_sem'] = (df_gp['retorno acumulado']/df_gp['retorno acumulado'].shift(pct).values -1) * 100

In [ ]:
def classe_gsrai(df, limit=0, media_movel=3):
    
    df=df.copy()
    df['mean'] = df['GSRAII Index'].rolling(media_movel).mean()
    up_down = df['mean']

    df['gsrai_gt_up'] = np.where((df['GSRAII Index'] > up_down) & (df['GSRAII Index'] >= limit), 1, 0)
    df['gsrai_gt_down'] = np.where((df['GSRAII Index'] < up_down) & (df['GSRAII Index'] >= limit), 1, 0)
    df['gsrai_lt_up'] = np.where((df['GSRAII Index'] > up_down) & (df['GSRAII Index'] < limit), 1, 0)
    df['gsrai_lt_down'] = np.where((df['GSRAII Index'] < up_down) & (df['GSRAII Index'] < limit), 1, 0)

    regras = ['gsrai_gt_up', 'gsrai_gt_down', 'gsrai_lt_up', 'gsrai_lt_down']
    df['classe'] = np.nan
    for col in regras:
        df.loc[df[col]==1, 'classe'] = col
    return df.dropna()

In [ ]:
df_gp =  classe_gsrai(df=df_gp, limit=0, media_movel=3)

In [ ]:
pct =  'pct_3sem'
regras = ['gsrai_gt_up', 'gsrai_gt_down', 'gsrai_lt_up', 'gsrai_lt_down']

In [ ]:
lista = []
for col in df_gp['classe'].unique():
    desc = df_gp[df_gp[col]==1][[pct]].describe().rename(columns={pct: col})
    lista.append(desc)
pd.concat(lista, axis=1)

In [ ]:
@interact(df=fixed(df_gp), 
          pct=['pct_1sem', 'pct_2sem', 'pct_3sem', 'pct_4sem', 'pct_5sem'],
          limit=FloatSlider(min=-1, max=1, step=0.05, value=0),
          media_movel=IntSlider(min=2, max=6, step=1, value=3)
         )
def hist_classes(df, pct, limit=0, media_movel=3):
    df = classe_gsrai(df=df, limit=limit, media_movel=media_movel)
    fig = px.histogram(df, x=pct, color="classe", marginal="box", nbins=100, 
#                        range_y=[0,50],
                       range_x=[df[pct].min()-1,df[pct].max()+1],
                       title=f'Distribuição {indice} {pct} ',
                      )
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.65)
    fig.add_vline(x=0,line_dash="dash")
    
    for i, classe in enumerate(df['classe'].unique()):
        fig.add_annotation(text=f'Total {classe}: {len(df[df["classe"]==classe])} ',
                           showarrow=False, yshift=70-(i*19), xshift=450)
        fig.add_annotation(text=f'σ {classe}: {df[df["classe"]==classe][pct].std():.2f} ',
                           showarrow=False, yshift=200-(i*19), xshift=450)
    
    return fig

    
hist_classes(df_gp, pct, limit=-1, media_movel=3);

EURO STOXX 50 | DAX

Pct: 1 sem	Limit: 0.00	Media_movel: 3

Qtd de registros: Balanceado

Divisão entre grupos: Os grupos UP apresentam mais de 75% dos valores acima de zero, ao passo que LT_DOWN apresenta mais de 75% abaixo de zero e GT_DOWN mais que 50% dos registros abaixo de zero. LT_DOWN e LT_UP apresenta outliers fortemente negativos e positivos, respectivamente. Os grupos LT apresentam mais volatilidade.

In [ ]:
indices = pd.DataFrame(columns=['Date'])

for indice in ['GDAXI', 'STOXX50E', 'FTSE']:

    url = f'https://query1.finance.yahoo.com/v7/finance/download/^{indice}?period1=852076800&period2=1634743473&interval=1d&events=history&includeAdjustedClose=true'
    df = pd.read_csv(url)
    df['Date'] = pd.to_datetime(df['Date'])
    df.rename(columns={'Close':indice}, inplace = True)
    df = df[['Date', indice]]
    indices = pd.merge(indices, df, how='right', on='Date')

indices.dropna(inplace = True)

indices.corr()

In [ ]:
indices.plot(x = 'Date', figsize=(20,10))